In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
!nvidia-smi

Tue Apr 07 14:05:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 442.19       Driver Version: 442.19       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 107... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   32C    P8     7W /  N/A |    111MiB /  8192MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
import os
import cv2
from time import time
import sys
sys.path.append('../plategen')
#sys.path.append('drive/My Drive/LPR/crnn')
#sys.path.append('drive/My Drive/LPR/plategen')
from plategen import TaiwanLicensePlateGenerator
    
now = time()

path = './test'
if not os.path.isdir(path):
    os.mkdir(path)
#g = TaiwanLicensePlateGenerator('drive/My Drive/LPR/plategen/char_black', 'drive/My Drive/LPR/plategen/char_s/-.jpg',
#                                'drive/My Drive/LPR/plategen/car_plate_border.png', True)
g = TaiwanLicensePlateGenerator('../plategen/char_black', '../plategen/char_s/-.jpg',
                                '../plategen/border/car_plate_border.png', True)
plates, labels = g.gen_plate(2800, 'perspective')
for plate, label in zip(plates, labels):
    cv2.imwrite(path+'/'+label+'.jpg', plate)
print(f'output to {path} for overall {time()-now:.2f} secs')

34 chars ['0', '1', '2', '3', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '-'] is loadeded
2,800 car plates is generated
output to ./test for overall 15.27 secs


In [5]:
from keras import backend as K
from keras.optimizers import Adadelta
from keras.callbacks import EarlyStopping, ModelCheckpoint
from Image_Generator import TextImageGenerator
#from Model_tf2 import get_Model
from Model import get_Model
from parameter import *
from imagegen import ImageGenerator
K.set_learning_phase(0)

# # Model description and training

model = get_Model(training=True)

try:
    #model.load_weights('drive/My Drive/LPR/crnn/model/lpr34+0407-270-3.384.hdf5')
    model.load_weights('model/wcp_0407_random+ep90+ep90-05-1.500.hdf5')
    print("...Previous weight data...")
except:
    print("...New weight data...")
    pass

train_size = 70000
train_batch_size = 192
tiger_train = ImageGenerator(None, img_w, img_h, train_batch_size, downsample_factor,
                             '../plategen/char_black',
                             '../plategen/char_s/-.jpg',
                             '../plategen/border/car_plate_border.png',
                             max_text_len)

valid_batch_size = 192
tiger_val = ImageGenerator('test/', img_w, img_h, valid_batch_size, downsample_factor, max_text_len=max_text_len)
tiger_val.build_data()

ada = Adadelta(lr=1.0)

early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=4, mode='min', verbose=1)
#checkpoint = ModelCheckpoint(filepath='drive/My Drive/LPR/crnn/model/lpr34+0407+ep270-{epoch:02d}-{val_loss:.3f}.hdf5',
#                             monitor='loss', verbose=1, mode='min', period=5)
checkpoint = ModelCheckpoint(filepath='model/wcp_0407_random+ep90+ep90+ep5-{epoch:02d}-{val_loss:.3f}.hdf5', monitor='loss', verbose=1, mode='min', period=1)
# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=ada)

# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    samples_per_epoch=int(train_size/train_batch_size),
                    #steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=600,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / valid_batch_size))

...Previous weight data...
2800  Image Loading start...
True
2800  Image Loading finish...


c:\users\enos\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:51: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., epochs=600, callbacks=[<keras.ca..., validation_data=<generator..., validation_steps=14, steps_per_epoch=364)`


Epoch 1/600
364/364 [==============================] - 486s 1s/step - loss: 74.2550 - val_loss: 40.4390

Epoch 00001: saving model to model/wcp_0407_random+ep90+ep90+ep5-01-40.439.hdf5
Epoch 2/600
 62/364 [====>.........................] - ETA: 8:47 - loss: 71.5903

KeyboardInterrupt: 